# SPISEA Quick Start: Making A Cluster

This is a quick start guide to making a synthetic cluster using the SPISEA package. The cluster is constructed using a user-specified isochrone and initial mass function (IMF). Detailed documentation is provided in the ReadtheDocs page (https://pypopstar.readthedocs.io/en/latest/).

Before starting this tutorial, it is assumed that SPISEA has been installed and the user's python path has been altered to include the SPISEA top-level directory

In [1]:
# Import necessary packages. 
from spisea import synthetic, evolution, atmospheres, reddening, ifmr
from spisea.imf import imf, multiplicity
import numpy as np
import pylab as py
import os
import glob
import pdb
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from specutils import Spectrum1D
import stsynphot as stsyn  
import astropy.units as u
from synphot import Observation
import pysynphot as S
global sig_int
sig_int = 0.1


Bad key "text.kerning_factor" on line 4 in
/Users/alexgagliano/miniconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
http://github.com/matplotlib/matplotlib/blob/master/matplotlibrc.template
or from the matplotlib source distribution
/Users/alexgagliano/miniconda3/lib/python3.7/site-packages/pysynphot/locations.py:345: UserWarning: Extinction files not found in /Users/alexgagliano/Documents/Research/2020oi/data/clusterFitting_SPISEA/models/grp/hst/cdbs/extinction
  warnings.warn('Extinction files not found in %s' % (extdir, ))


In [2]:
S.ObsBandpass('acs,wfc2,f555w')
#S.ObsBandpass('acs,wfc2,f439w')
#S.ObsBandpass('acs,wfc2,f702w')
#S.ObsBandpass('acs,wfc2,f218w')
S.ObsBandpass('acs,hrc,f555w')
S.ObsBandpass('acs,hrc,f814w')
S.ObsBandpass('acs,hrc,f330w')
#S.ObsBandpass('acs,wfc2,f380w')
#S.ObsBandpass('acs,wfc2,f791')
S.ObsBandpass('acs,wfc2,f775w')
S.ObsBandpass('acs,wfc2,f475w')
#S.ObsBandpass('acs,wfc2,f555w')
#S.ObsBandpass('acs,wfc2,f814w')
S.ObsBandpass('wfc3,ir,f160w')
#S.ObsBandpass('acs,wfc2,f814w')

In [3]:
#S.ObsBandpass('acs,wfc2,f555w')
#S.ObsBandpass('acs,hrc,f555w')
#S.ObsBandpass('acs,hrc,f814w')
#S.ObsBandpass('acs,hrc,f330w')
#S.ObsBandpass('acs,wfc2,f775w')
#S.ObsBandpass('acs,wfc2,f475w')
#S.ObsBandpass('wfc3,ir,f160w')

In [4]:
import pandas as pd
HST_20oi = pd.read_csv("/Users/alexgagliano/Documents/Research/2020oi/data/photometry/HST_preExplosionPhotometry_dataOnly.csv", delim_whitespace=True)
HST_20oi = HST_20oi.drop_duplicates(subset=['Instrument', 'Filter'])

f814_hrc = HST_20oi.loc[HST_20oi['Instrument'].isin(['ACS/HRC']) & HST_20oi['Filter'].isin(['F814W']), 'Magnitude'].values[0]
f555_hrc = HST_20oi.loc[HST_20oi['Instrument'].isin(['ACS/HRC']) & HST_20oi['Filter'].isin(['F555W']), 'Magnitude'].values[0]
f330_hrc = HST_20oi.loc[HST_20oi['Instrument'].isin(['ACS/HRC']) & HST_20oi['Filter'].isin(['F330W']), 'Magnitude'].values[0]
f160_ir = HST_20oi.loc[HST_20oi['Instrument'].isin(['WFC3/IR']) & HST_20oi['Filter'].isin(['F160W']), 'Magnitude'].values[0]
f555_wfc2 = HST_20oi.loc[HST_20oi['Instrument'].isin(['WFC3/UVIS']) & HST_20oi['Filter'].isin(['F555W']), 'Magnitude'].values[0]
f775_wfc2 = HST_20oi.loc[HST_20oi['Instrument'].isin(['WFC3/UVIS']) & HST_20oi['Filter'].isin(['F775W']), 'Magnitude'].values[0]
f475_wfc2 = HST_20oi.loc[HST_20oi['Instrument'].isin(['WFC3/UVIS']) & HST_20oi['Filter'].isin(['F475W']), 'Magnitude'].values[0]

f814_hrc_err = HST_20oi.loc[HST_20oi['Instrument'].isin(['ACS/HRC']) & HST_20oi['Filter'].isin(['F814W']), 'Uncertainty'].values[0]
f555_hrc_err = HST_20oi.loc[HST_20oi['Instrument'].isin(['ACS/HRC']) & HST_20oi['Filter'].isin(['F555W']), 'Uncertainty'].values[0]
f330_hrc_err = HST_20oi.loc[HST_20oi['Instrument'].isin(['ACS/HRC']) & HST_20oi['Filter'].isin(['F330W']), 'Uncertainty'].values[0]
f160_ir_err = HST_20oi.loc[HST_20oi['Instrument'].isin(['WFC3/IR']) & HST_20oi['Filter'].isin(['F160W']), 'Uncertainty'].values[0]
f555_wfc2_err = HST_20oi.loc[HST_20oi['Instrument'].isin(['WFC3/UVIS']) & HST_20oi['Filter'].isin(['F555W']), 'Uncertainty'].values[0]
f775_wfc2_err = HST_20oi.loc[HST_20oi['Instrument'].isin(['WFC3/UVIS']) & HST_20oi['Filter'].isin(['F775W']), 'Uncertainty'].values[0]
f475_wfc2_err = HST_20oi.loc[HST_20oi['Instrument'].isin(['WFC3/UVIS']) & HST_20oi['Filter'].isin(['F475W']), 'Uncertainty'].values[0]

#### Step 1: Make a SPISEA isochrone object

The cluster is made from a theoretical isochrone at a given age, extinction, and distance from Earth. These parameters MUST be specified by the user. Other inputs (e.g. stellar evolution/atmosphere models, extinction law, and photometric filters used) are optional keywords. See documentation for all keywords and their default values.

Important Note: The IsochronePhot class saves its output as a FITS table, which it will read on subsequent calls for the same isochrone rather than regenerating it from scratch. We highly recommend reading the "Tips and Tricks: The IsochronePhot Object" section of the Isochrone object documentation for details on how this process works.

Here, we create a 100 Myr cluster isochrone at an extinction of E(B-V)=0.174 and distance of 10^7 pc from Earth (the distance to M100). 

In [5]:
age_init = 7. #age in log10(years)
mass_init = 4. #mass in log10(Msol)
Z_init = 0
theta_init = [age_init, mass_init, Z_init]

# the log-likelihood for all galaxies simultaneously
def neg_log_likelihood(theta, obs, obs_sig):    
    chisq = np.zeros(len(obs_sig))
    for i in np.arange(len(chisq)):
        chisq += (fit(theta)[i] - obs[i])**2/(obs_sig[i]**2+sig_int**2)
    return 0.5*np.sum(chisq)

# only if the prior makes sense for all vals simultaneously do we 
# accept it
def log_prior(theta):
    #prior = [0., 0., 0.]
    age = theta[0]
    mass = theta[1]  
    metallicity = theta[2]
    logprior_age = logprior_mass = logprior_metallicity = -np.inf
    
    if (6<=age<=10) and (2.5<=mass<=6.5) and (-0.5<metallicity<0.5):
        logprior_age=logprior_mass=logprior_metallicity = 0
    prior = logprior_age + logprior_mass + logprior_metallicity
    return np.sum(prior)

# the posterior is the combination of the prior and the likelihood
def log_posterior(theta, obs, obs_sig):
    return log_prior(theta)+neg_log_likelihood(theta, obs, obs_sig)

In [6]:
def fit(theta):
    fileList = glob.glob('./iso_*.fits', recursive=True)
    for file in fileList:
        os.remove(file)
    logAge = theta[0]
    mass = theta[1]
    # Define isochrone parameters
    EBV = 0.174
    AV = 3.2*EBV
    dist = 1.71e7 #distance to M100 in parsec
    #metallicity = -0.096 #(Metallicity in [M/H] (this should be 80% solar metallicity)
    metallicity = theta[2] #(Metallicity in [M/H] (this should be 80% solar metallicity)
    evo_model = evolution.MISTv1() 
    atm_func = atmospheres.get_merged_atmosphere
    red_law = reddening.RedLawHosek18b()
    #filt_list = ['wfc3,ir,f160w', 'acs,wfc1,f814w']

    # Make Isochrone object. Note that is calculation will take a few minutes, unless the 
    # isochrone has been generated previously.
    my_iso = synthetic.IsochronePhot(logAge, AV, dist, metallicity=metallicity,
                                evo_model=evo_model, atm_func=atm_func,
                                red_law=red_law, filters=[]) #remove the filter list for now

        # Make multiplicity object
    imf_multi = multiplicity.MultiplicityUnresolved()

    # Make IMF object; we'll use a broken power law with the parameters from Kroupa+01.

    # NOTE: when defining the power law slope for each segment of the IMF, we define
    # the entire exponent, including the negative sign. For example, if dN/dm $\propto$ m^-alpha,
    # then you would use the value "-2.3" to specify an IMF with alpha = 2.3. 
    #
    # Here we define a Kroupa IMF using the Multiplicity properties defined in Lu+13. 
    #
    massLimits = np.array([0.2, 0.5, 1, 120]) # Define boundaries of each mass segement
    powers = np.array([-1.3, -2.3, -2.3]) # Power law slope associated with each mass segment
    my_imf = imf.IMF_broken_powerlaw(massLimits, powers, imf_multi)
    mass = 10**mass #range from 10**2.5 to 10**6.5 
    cluster = synthetic.UnresolvedCluster(my_iso, my_imf, mass, wave_range=[1000, 52000],verbose=True)
    spec = S.ArraySpectrum(cluster.wave_trim, cluster.spec_trim)
    #sns.set_context("talk")
    #plt.plot(cluster.wave_trim, cluster.spec_trim)
    #plt.xlabel("Wave (AA)")
    #plt.ylabel(r"$F_{\lambda}$")
    #plt.yscale("log")

    obs814_hrc =S.Observation(spec,S.ObsBandpass('acs,hrc,f814w')).effstim('abmag')
    obs555_hrc =S.Observation(spec,S.ObsBandpass('acs,hrc,f555w')).effstim('abmag')
    obs330_hrc =S.Observation(spec,S.ObsBandpass('acs,hrc,f330w'),  force='extrap').effstim('abmag')    
    obs160_ir = S.Observation(spec,S.ObsBandpass('wfc3,ir,f160w')).effstim('abmag')
    obs555_wfc2 = S.Observation(spec,S.ObsBandpass('acs,wfc2,f555w')).effstim('abmag')
    obs775_wfc2 = S.Observation(spec,S.ObsBandpass('acs,wfc2,f775w')).effstim('abmag')
    obs475_wfc2 = S.Observation(spec,S.ObsBandpass('acs,wfc2,f475w')).effstim('abmag')

    return np.array([obs814_hrc, obs555_hrc, obs330_hrc, obs160_ir, obs555_wfc2, obs775_wfc2, obs475_wfc2])

In [ ]:
import emcee
N = 100
pos = theta_init + 1.e-1*np.random.randn(N, 3)
obs_sig = np.array([f814_hrc_err, f555_hrc_err, f330_hrc_err, f160_ir_err, f555_wfc2_err, f775_wfc2_err, f475_wfc2_err])
obs = np.array([f814_hrc, f555_hrc, f330_hrc, f160_ir, f555_wfc2, f775_wfc2, f475_wfc2])

nwalkers, ndim = pos.shape
sampler = emcee.EnsembleSampler(nwalkers, ndim, log_posterior, args=(obs, obs_sig))
sampler.run_mcmc(pos, N, progress=True);

  2%|▏         | 2/100 [21:58:56<1156:43:57, 42492.22s/it]

In [ ]:
fig, axes = plt.subplots(2, figsize=(10, 7), sharex=True)
samples = sampler.get_chain()

labels = [r"Age", "Mass"]
for i in range(2):
    ax = axes[i]
    ax.plot(samples[:, :, i], "k", alpha=0.3)
    ax.set_xlim(0, len(samples))
    ax.set_ylabel(labels[i])
    ax.yaxis.set_label_coords(-0.1, 0.5)
axes[-1].set_xlabel("step number");

In [ ]:
# show the full corner plot 
import corner
samples_postBurnIn = samples[5:,:,:]
samples_post = samples_postBurnIn.reshape((-1, 2))
fig = corner.corner(samples_post, labels=labels,smooth=True);


Once calculated, the isochrone will be written as a fits file to a location set by the "iso_dir" keyword (not shown here; default location is current working directory). In the future, the IsochronePhot function will read this file directly rather than recalculating the isochrone again. 

The file name will be the following: "iso_logAge_AKs_distance_metallicity.fits, using the specified values

### Step 3: Make the Cluster  
#### Option 1: No compact objects
To create the cluster, the user passes in an isochrone object, and imf object, and specifies the total cluster mass. Here we will make a 10^4 M_sun cluster using the isochrone and imf we have defined.

The individual star systems in the cluster and their properties are stored in an astropy table accessed by the star_systems subfunction on the cluster object. Note that the photometry of these star systems includes the contributions from all companions, which are generated probabilistically using the multiplicity defined in the IMF object.

In [ ]:
import seaborn as sns
sns.set_context("talk")
plt.plot(cluster.wave_trim, cluster.spec_trim)
plt.xlabel("Wave (AA)")
plt.ylabel(r"$F_{\lambda}$")
plt.yscale("log")

In [ ]:

#sp = Spectrum1D(spectral_axis=S.Vega.wave, flux=S.Vega.flux)
test = S.ArraySpectrum(cluster.wave_trim, cluster.spec_trim)
obs160 = S.Observation(test,S.ObsBandpass('wfc3,ir,f160w'))
#obs160.effstim('stmag')
obs814 = S.Observation(test,S.ObsBandpass('acs,wfc1,f814w'))
#obs814.effstim('stmag')

In [ ]:
# Look at the cluster CMD, compared to input isochrone. Note the impact of
# multiple systems on the photometry
#clust = cluster.star_systems
#iso = my_iso.points
#plt.plot(obs160.effstim('stmag'),obs814.effstim('stmag'), '*', label='simulated')
plt.plot(obs160.effstim('abmag'),obs814.effstim('abmag'), '*', label='simulated')
py.errorbar(F160_20oi, F814_20oi, yerr=F814err_20oi, xerr=F160err_20oi, fmt='o', label='obs')
#py.plot(HST_20oi.loc[HST_20oi['Filter'] == 'F814W', 'Limit'].values[0] - HST_20oi.loc[HST_20oi['Filter'] == 'F160W', 'Limit'].values[0], HST_20oi.loc[HST_20oi['Filter'] == 'F814W', 'Limit'].values[0], 'o', label='obs')
py.xlabel('F160M')
py.ylabel('F814W')
py.gca().invert_yaxis()
py.legend();

In [ ]:
HST_20oi['Limit']